In [15]:
!pip3 install chromadb

  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached fastapi-0.115.4-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.32.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached posthog-3.7.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached monotonic-1.6-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4

In [ ]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
import getpass
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import warnings
warnings.filterwarnings("ignore")
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from IPython.display import Image, display
import pprint
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing import Annotated, Literal
from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
import ast


def build_vector_db():

    def _set_env():
        load_dotenv()
        if "GOOGLE_API_KEY" not in os.environ:
            os.environ["GOOGLE_API_KEY"] = getpass.getpass(
                "Enter your Google AI API key: "
            )

    _set_env()

    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        # other params...
    )

    urls = [
        os.environ["moneycontrol"],
        os.environ["economic_times"],
        os.environ["yahoo_fin"],
    ]

    print(urls)

    docs = [WebBaseLoader(url).load() for url in urls]
    docs_list = [item for sublist in docs for item in sublist]

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=100, chunk_overlap=20
    )
    doc_splits = text_splitter.split_documents(docs_list)

    # Add to vectorDB
    vectorstore = Chroma.from_documents(
        documents=doc_splits,
        collection_name="rag-chroma",
        embedding=embeddings,
        
    )
    retriever = vectorstore.as_retriever()

    retriever_tool = create_retriever_tool(
        retriever,
        "retriver_stock_market_updates",
        "Search and return information about stocks in news whether they are good news or bad news , , bullish and bearnish newses as well",
    )

    tools = [retriever_tool]

    return retriever_tool, retriever, tools, vectorstore


/home/kgvt/project_agent_final/project_agent/project_agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [16]:
retriever_tool, retriever, tools, vectorstore = build_vector_db()

['https://www.moneycontrol.com/', 'https://economictimes.indiatimes.com/markets/stocks?from=mdr', 'https://finance.yahoo.com/']


In [19]:
persist_directory = "project_agent_final/project_agent/vector_db"

In [20]:
persist_directory

'project_agent_final/project_agent/vector_db'

In [24]:

collection = persistent_client.get_or_create_collection("agent")



NameError: name 'persistent_client' is not defined

In [22]:
import chromadb


# Initialize ChromaDB client with persistence
client = chromadb.Client(
    persist_directory="project_agent_final/project_agent/vector_db"
)

TypeError: Client() got an unexpected keyword argument 'persist_directory'

In [ ]:
chromadb.Client()